# <center>Advanced ML pipeline with segmentation_models and Callbacks (W7S1)

## **The most important thing to improve your model performance is to understand each and every step taken to build the final model.** 

*In this notebook, you will see how we increase our accuracy from 0.18 in previous lecture to 0.8 in this lecture.*


# Importing libraries

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

import cv2
import keras
from tqdm import tqdm

import tensorflow as tf
import glob
from PIL import Image
from sklearn.model_selection import train_test_split

from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math
from tensorflow.keras.utils import to_categorical, Sequence


import datetime

In [2]:
'''
Here load_data function is called. This will load the dataset paths and 
split it into X_train, X_test, y_train, y_test '''

img_dir = '../input/artificial-lunar-rocky-landscape-dataset/images/render'
mask_dir = '../input/artificial-lunar-rocky-landscape-dataset/images/clean'


# let's get the list of image paths and mask paths in sorted order from the given directory respectively
images = [os.path.join(img_dir, x) for x in sorted(os.listdir(img_dir))]
masks = [os.path.join(mask_dir, x) for x in sorted(os.listdir(mask_dir))]


# in this session, we will use our complete dataset
X_train = images[:8000]
y_train = masks[:8000]

X_valid = images[8000:]
y_valid = masks[8000:]

## Building Dataset generator from scratch using Sequence

In [3]:
# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence
class LunarDataset(Sequence):

    def __init__(self, x_set, y_set, batch_size, dims, classes):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.img_height, self.img_width = dims
        self.classes = classes


    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        count = 0
        # https://numpy.org/doc/stable/reference/generated/numpy.zeros.html
        xtr = np.zeros((self.batch_size, self.img_height, self.img_width, 3))
        for filename in batch_x:
            img = imread(filename)[:self.img_height, :self.img_width, :] / 255.0
            img = img.astype(np.float32)
            xtr[count] = img
            count += 1
            
        count = 0
        ytr = np.zeros((self.batch_size, self.img_height, self.img_width, num_classes))
        for filename in batch_y:
            mask = imread(filename, as_gray = True)[:self.img_height, :self.img_width] // 0.07
            mask[mask == 3] = 2
            mask[mask == 10] = 3
            
            # one hot encoding our masks using to_categorical
            # https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
            mask = to_categorical(mask, num_classes = 4)
            ytr[count] = mask
            count += 1

        return xtr, ytr.astype(np.int32)


batch_size = 16
dims = (480, 480)
num_classes = 4

train_dataset = LunarDataset(X_train, y_train, batch_size, dims, num_classes)
valid_dataset = LunarDataset(X_valid, y_valid, batch_size, dims, num_classes)

## Let's visualize our masks

In [4]:
sam = next(iter(train_dataset))

sample = sam[1][1]

i, v = np.unique(sample, return_counts = True)
for a,b in zip(i,v):
    print(a," ", b)
    

fig, (a1, a2, a3, a4) = plt.subplots(1, 4, figsize = (20, 5))

a1.imshow(sample[:, :, 0])
a2.imshow(sample[:, :, 1])
a3.imshow(sample[:, :, 2])
a4.imshow(sample[:, :, 3])

plt.show()

# four channels showing different classes
# each channel have only 0 and 1 values

### Let's check out some basic steps of transfer learning using a pretrained model (VGG16)


In [5]:
#### Step 1: Creating a base model 

IMG_SHAPE = (480, 480, 3)

# include_top specify that we don't want to use the top layer (classifier)
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')




#### Step 2: Freezing the base

# It is important to freeze the convolutional base before you compile and train the model.
# Freezing prevents the weights in a given layer from being updated during training
# VGG16 has many layers, so setting the entire model's trainable flag to False will freeze all of them.

base_model.trainable = False

# Let's take a look at the base model architecture
base_model.summary()



#### Step 3: Adding the head

# inputs
inputs = tf.keras.Input(shape=(480, 480, 3))

# base with pretrained model
x = base_model(inputs, training=False)

# head layers
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(2)(x)

# model
model = tf.keras.Model(inputs, outputs)

# Let's take a look at the final model architecture
model.summary()


# reference: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## segmentation_models


**segmentation_models** is a python library with Neural Networks for Image Segmentation based on Keras and TensorFlow.

The main features of this library are:

* High level API (just two lines of code to create model for segmentation)
* 4 models architectures for binary and multi-class image segmentation (including legendary Unet)
* 25 available backbones for each architecture
* All backbones have pre-trained weights for faster and better convergence
* Helpful segmentation losses (Jaccard, Dice, Focal) and metrics (IoU, F-score)

In [6]:
# run this command to directly install the library in our notebook

!pip install segmentation_models

* Provide environment variable SM_FRAMEWORK=keras / SM_FRAMEWORK=tf.keras before import segmentation_models
* Change framework sm.set_framework('keras') / sm.set_framework('tf.keras')

In [8]:
# By default it tries to import keras, if it is not installed, it will try to start with tensorflow.keras framework

import segmentation_models as sm
os.environ["SM_FRAMEWORK"] = "tf.keras"
sm.set_framework('tf.keras')
tf.keras.backend.set_image_data_format('channels_last')

# Building our UNet model with segmentation_models

In [9]:
BACKBONE = 'vgg16'
input_shape = (480, 480, 3)
n_classes = 4
activation = 'softmax'

# using segmentation_models to create U-net with vgg16 as a backbone
# and pretrained imagenet weights

# segmentation_model basically will create a mirror image of our backbone as expansion path and add to the contraction path
model = sm.Unet(backbone_name = BACKBONE, 
                input_shape = input_shape, 
                classes = n_classes, 
                activation = activation,
                encoder_weights = 'imagenet')
model.summary()

## Compile model

In [10]:
""" Hyperparameters """
lr = 1e-4
batch_size = 16
epochs = 1

# metrics for result validation
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compiling the model
model.compile(loss = 'categorical_crossentropy', 
              optimizer = tf.keras.optimizers.Adam(lr), 
              metrics = metrics)

train_steps = len(X_train)//batch_size
valid_steps = len(X_valid)//batch_size


""" Callbacks """
current_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks
callbacks = [
        tf.keras.callbacks.ModelCheckpoint(filepath=f'models/lunarModel_{current_datetime}.h5',
                        monitor='val_iou_score', verbose=0, 
                        mode='max', save_best_model=False),
             
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_iou_score", mode='max', patience=4,
                          factor=0.1, verbose=0, min_lr=1e-6),
             
        tf.keras.callbacks.EarlyStopping(monitor="val_iou_score", patience=5, verbose=0, mode='max'),

        tf.keras.callbacks.TensorBoard(f'models/logs_{current_datetime}')
    ]


model is compiled with **loss**="categorical_crossentropy",  **optimizer**=Adam, **metrics**=iou_score

**Callbacks** is a tool to customize the behavior of a Keras model during training, evaluation, or inference.

**ModelCheckpoint:** used  to periodically save your model during training.

**ReduceLROnPlateau:** Reduce learning rate when a metric has stopped improving.

**EarlyStopping:** Stop training when a monitored metric has stopped improving.

**TensorBoard:** Enable visualizations for TensorBoard.

## Train model

In [16]:
# Fitting the model
model_history = model.fit(train_dataset,
        steps_per_epoch=train_steps,
        validation_data=valid_dataset,
        validation_steps=valid_steps,
        epochs=epochs,
        callbacks=callbacks
    )
# val_iou_score is expected to reach almost 0.8 after 5 epochs even without using callbacks

Epoch 1/5
500/500 [==============================] - 553s 1s/step - loss: 0.4765 - iou_score: 0.4762 - f1-score: 0.5403 - val_loss: 0.2147 - val_iou_score: 0.5423 - val_f1-score: 0.5970
Epoch 2/5
500/500 [==============================] - 508s 1s/step - loss: 0.1501 - iou_score: 0.6948 - f1-score: 0.7837 - val_loss: 0.1720 - val_iou_score: 0.6603 - val_f1-score: 0.7461
Epoch 3/5
500/500 [==============================] - 512s 1s/step - loss: 0.1204 - iou_score: 0.7489 - f1-score: 0.8338 - val_loss: 0.1116 - val_iou_score: 0.7383 - val_f1-score: 0.8242
Epoch 4/5
500/500 [==============================] - 508s 1s/step - loss: 0.1053 - iou_score: 0.7772 - f1-score: 0.8573 - val_loss: 0.1160 - val_iou_score: 0.7461 - val_f1-score: 0.8315
Epoch 5/5
500/500 [==============================] - 514s 1s/step - loss: 0.0989 - iou_score: 0.7877 - f1-score: 0.8656 - val_loss: 0.0906 - val_iou_score: 0.7976 - val_f1-score: 0.8726

## Predict from model

In [11]:
# function to predict result 
def predict_image(img_path, mask_path, model):
    H = 480
    W = 480
    num_classes = 4

    img = imread(img_path)
    img = img[:480, :480, :]
    img = img / 255.0
    img = img.astype(np.float32)

    ## Read mask
    mask = imread(mask_path, as_gray = True)
    mask = mask[:480, :480]
    
    ## Prediction
    pred_mask = model.predict(np.expand_dims(img, axis=0))
    pred_mask = np.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[0]
    
    
    # calculating IOU score
    inter = np.logical_and(mask, pred_mask)
    union = np.logical_or(mask, pred_mask)
    
    iou = inter.sum() / union.sum()

    return img, mask, pred_mask, iou

In [12]:
img_path = '../input/artificial-lunar-rocky-landscape-dataset/images/render/render0042.png'
mask_path = '../input/artificial-lunar-rocky-landscape-dataset/images/clean/clean0042.png'

img, mask, pred_mask, iou = predict_image(img_path, mask_path, model)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15, 10))

ax1.set_title("Input Image")
ax1.imshow(img)

ax2.set_title("True Mask")
ax2.imshow(mask, cmap = "gray")

ax3.set_title("Predicted mask with IOU score %.2f"%(iou))
ax3.imshow(pred_mask, cmap = "gray")

plt.show()

### You can see we have reached a satisfactory result for our project. 

## Can we load a saved model in a new session and use it for predictions - WITHOUT TRAINING AGAIN?
Yes, if we download it before shutting down our current kernel.

On starting the new session, we can upload the saved model and use it for initializing weights of a newly created model instance!

This is a new model 'variable' with optimized weights! - Kinda like our very own pretrained model :)

In [13]:
# create a new model instance
new_model = sm.Unet(backbone_name = BACKBONE, 
                input_shape = input_shape, 
                classes = n_classes, 
                activation = activation,
                encoder_weights = 'imagenet')

# define the path to the checkpointed model after uploading it
checkpoint_path = '../input/savedmodel1/lunarModel_20220530-061940.h5'

#resolve dependencies - sometimes..
dependencies = {
    'iou_score': sm.metrics.IOUScore,
    'f1-score': sm.metrics.FScore
}

# load the weights from the checkpoint
new_model = keras.models.load_model(checkpoint_path, custom_objects=dependencies)
 
# use loaded info for prediction
img_path = '../input/artificial-lunar-rocky-landscape-dataset/images/render/render0028.png'
mask_path = '../input/artificial-lunar-rocky-landscape-dataset/images/clean/clean0028.png'

img, mask, pred_mask, iou = predict_image(img_path, mask_path, new_model)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (15, 10))

ax1.set_title("Input Image")
ax1.imshow(img)

ax2.set_title("True Mask")
ax2.imshow(mask, cmap = "gray")

ax3.set_title("Predicted mask with IOU score %.2f"%(iou))
ax3.imshow(pred_mask, cmap = "gray")

plt.show()



## In next lecture, you will learn some best practices to optimize your models